<a href="https://colab.research.google.com/github/Abhishek0697/Detection-of-Hate-Speech-in-Multimodal-Memes/blob/main/Code/Experiments/BERT%2BResNeXt%2BImage_Captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_ids = [0]
device = torch.device('cuda:'+ str(gpu_ids[0]) if torch.cuda.is_available() else 'cpu')

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from dataloader import mydataset_captioning, mytestdataset
from Load_model import load
from plot_curves import plot_loss, plot_acc
from trainer_Resnet_BERT_Captioning import train, test_classify
from resnet_models import ResNet,Bottleneck, resnext101_32x8d
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

In [ ]:
trainlist = r'Sarcasm.tsv'
validlist = r'test.tsv'

In [ ]:
train_dataset = mydataset_captioning(trainlist, name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 4, num_workers=16,pin_memory=True)
validation_dataset = mydataset_captioning(validlist, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 4, num_workers=16,pin_memory=True)

Streaming output truncated to the last 5000 lines.
https://www.huffingtonpost.com/entry/trump-organization-resigning-conflicts_us_58867ec1e4b096b4a23417db	trump letter resigning from hundreds of companies seems like a big deal. it isn't.	0

https://www.theonion.com/aquarium-unveils-floating-carcasses-of-the-pacific-exhi-1819574735	aquarium unveils 'floating carcasses of the pacific' exhibit	1

https://entertainment.theonion.com/early-humans-finally-drunk-enough-to-invent-dancing-1819571222	early humans finally drunk enough to invent dancing	1

https://www.huffingtonpost.com/entry/colorados-new-revenge-por_b_5427703.html	colorado's new revenge porn statute is good law and sound policy	0

https://www.huffingtonpost.com/entry/donate-eggs_b_5884734.html	i donated my eggs so i could travel the world	0

https://www.huffingtonpost.com/entry/hack-election-russia_us_5952cacce4b0da2c731f6090	we must ensure democratic integrity in the digital age	0

https://www.theonion.com/microlender-forecloses

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg

https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5	former versace store clerk sues over secret 'black code' for minority shoppers	0

https://www.huffingtonpost.com/entry/roseanne-revival-review_us_5ab3a497e4b054d118e04365	the 'roseanne' revival catches up to our thorny political mood, for better and worse	0

https://local.theonion.com/mom-starting-to-fear-son-s-web-series-closest-thing-she-1819576697	mom starting to fear son's web series closest thing she will have to grandchild	1

https://politics.theonion.com/boehner-just-wants-wife-to-listen-not-come-up-with-alt-1819574302	boehner just wants wife to listen, not come up with alternative debt-reduction ideas	1

https://www.huffingtonpost.com/entry/jk-rowling-wishes-snape-happy-birthday_us_569117c4e4b0cad15e64fdcb	j.k. rowling wishes snape happy birthday in the most magical way	0

https://www.huffingtonpost.com/entry/advancing-the-worlds-women_b_6810038.html	advancing the world's women	0

https://www.hu

**Model Definition**

In [ ]:
'''
Model 2 BERT

Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
''' 

Text_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 2,   
    output_attentions = False, 
    output_hidden_states = True
)

Text_model = nn.DataParallel(Text_model,device_ids=gpu_ids).to(device)


'''
Load saved model from checkpoint
'''
#model2_name = 'BERT_basic'
#model2_path = './saved_model_checkpoints/'+model2_name

#checkpoint2 = torch.load(model2_path)
#Text_model.load_state_dict(checkpoint2['model_state_dict'])

#Text_model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

'\nLoad saved model from checkpoint\n'

In [ ]:
class FusionNet(nn.Module):
    
    def __init__(self, num_classes, drop_prob = 0.1):
        super(FusionNet, self).__init__()
        
        self.concat = nn.Linear(in_features=768, out_features= 512)
        
        self.bn = nn.BatchNorm1d(512)
        self.bn1 = nn.BatchNorm1d(768)
        self.bn2 = nn.BatchNorm1d(2048)
        self.bn3 = nn.BatchNorm1d(768)

        
        self.dropout = nn.Dropout(drop_prob)
        
        self.classify = nn.Linear(in_features = 512, out_features = num_classes)
        
        
    def forward(self, text_features):

        text_features = self.bn1(text_features)
        #image_features = self.bn2(image_features)
        #caption_features = self.bn3(caption_features)

        #fused_input =  torch.cat((text_features, image_features, caption_features), dim=1)
        
        x = self.concat(text_features)
        x = F.relu(self.bn(x))        
        
        y = self.classify(x) 

        return x,y


In [ ]:
Fusion_model = FusionNet(num_classes = 1 , drop_prob = 0.1)
Fusion_model = nn.DataParallel(Fusion_model, device_ids=gpu_ids).to(device)

**Hyperparameters**

In [ ]:

criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.SGD(Fusion_model.parameters(), lr=0.1, weight_decay=1e-4, momentum=0.9)
# optimizer = AdamW(Fusion_model.parameters(), lr = 2e-3, eps = 1e-8)


'''
Number of training epochs.
'''
num_Epochs = 1


# '''
# OneCycleLR
# '''
# max_lr = 0.05
# lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, total_steps=None, epochs=num_Epochs, steps_per_epoch=len(train_dataloader), pct_start=0.3, anneal_strategy='cos', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=25.0, final_div_factor=10000.0, last_epoch=-1)


lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 4, gamma = 0.001)

# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1)

In [ ]:
model_name = 'SarcasmNewsHeadline'
model_path = 'saved_model_checkpoints/'+model_name

In [ ]:
model_path

'saved_model_checkpoints/SarcasmNewsHeadline'

In [ ]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import defaultdict
svm_train = []
svm_t = []

train_loss= []
v_loss = []
v_acc = []


def train_s(text_model,fusion_model,data_loader,test_loader,criterion,optimizer, lr_scheduler, modelpath, writer, device, epochs):
    
    fusion_model.train()

    for epoch in range(epochs):
        avg_loss = 0.0
                

        for batch_num, (captions, input_ids, attention_masks, target) in enumerate(data_loader):
            
            target = target.to(device)
            input_ids, attention_masks = input_ids.to(device), attention_masks.to(device)
          #  input_ids_cap, attention_masks_cap = input_ids_cap.to(device), attention_masks_cap.to(device)

            
            '''
            Compute ResNet Features
            '''
     #       out, image_features = image_model(feats)
           
                            
            '''
            Compute BERT Features for actual captions
            Take hidden state corresponding to [CLS] token from the final transformer
            '''
            output_dictionary = text_model(input_ids, 
                                           token_type_ids=None, 
                                           attention_mask=attention_masks, 
                                           labels=target,
                                           return_dict = True)
            
            text_features = output_dictionary.hidden_states[12][:,0,:]
            
            
                     
            
            '''
            Compute BERT Features for captions generated by Image Captioning Model
            '''
       #     output_dictionary_captioning = text_model(input_ids_cap, 
       #                                    token_type_ids=None, 
       #                                    attention_mask=attention_masks_cap, 
       #                                    labels=target,
       #                                    return_dict = True)
       #     
       #     captioning_features = output_dictionary_captioning.hidden_states[12][:,0,:]

            
            
       
             
            '''
            Compute Classification Output and loss from Fusion model
            '''
            yy,output = fusion_model(text_features)
            target = target.float()
            svm_train.append(yy)
            svm_t.append(target)
            loss = criterion(output, target.unsqueeze(1))
            print("okay ",batch_num)
            #print("okay ",output)
            #print("okay ",target)

            
                   
            '''
            Take Step
            '''                    
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            avg_loss += loss.item()


#             '''
#             linear_schedule_with_warmup take step after each batch
#             '''
#             lr_scheduler.step()
            
                                        
           # del feats
            del captions
            del input_ids
            del attention_masks
           # del input_ids_cap
            #del attention_masks_cap
            del target
            del loss
            
            
        training_loss = avg_loss/len(data_loader)
       
        print('Epoch: ', epoch+1)            
        print('training loss = ', training_loss)
        train_loss.append(training_loss)

        
        
        '''
        Learning rate scheduler
        '''
        lr_scheduler.step()
            
            
        '''
        Check performance on validation set after an Epoch
        '''
        
        valid_loss, top1_acc= test_classify(image_model, text_model, fusion_model, test_loader, criterion, device)
        print('Validation Loss: {:.4f}\tTop 1 Validation Accuracy: {:.4f}'.format(valid_loss, top1_acc))
        v_loss.append(valid_loss)
        v_acc.append(top1_acc)

        '''
        Logs
        '''
        writer.add_scalar("Loss/train", training_loss, epoch)            
        writer.add_scalar('Loss/Validation', valid_loss, epoch)
        writer.add_scalar('Accuracy/Validation', top1_acc, epoch)
               
        '''
        save fusion model checkpoint after every epoch
        '''
        torch.save({
            'model_state_dict': fusion_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'Training_Loss_List':train_loss,
            'Validation_Loss_List':v_loss,
            'Validation_Accuracy_List': v_acc,
            'Epoch':epoch,
            'lr_scheduler': lr_scheduler.state_dict() 

            }, modelpath)


        
'''
Returns Loss and top1 accuracy on validation set
'''
def test_classify(text_model, fusion_model, test_loader, criterion, device):
    fusion_model.eval()
    test_loss = []
    top1_accuracy = 0
    total = 0

    for batch_num, (captions, input_ids, attention_masks, target) in enumerate(test_loader):
        
        target = target.to(device)
        input_ids, attention_masks = input_ids.to(device), attention_masks.to(device)
        #input_ids_cap, attention_masks_cap = input_ids_cap.to(device), attention_masks_cap.to(device)

        
        '''
        Compute ResNet Features
        '''
      #  out, image_features = image_model(feats) 

        
        
        '''
        Compute BERT Features
        '''
        output_dictionary = text_model(input_ids, 
                                       token_type_ids=None, 
                                       attention_mask=attention_masks, 
                                       labels=target,
                                       return_dict = True)

        text_features = output_dictionary.hidden_states[12][:,0,:]
        
        
        
        '''
        Compute BERT Features for captions generated by Image Captioning Model
        '''
       # output_dictionary_captioning = text_model(input_ids_cap, 
     ##                                  token_type_ids=None, 
      #                                 attention_mask=attention_masks_cap, 
      #                                 labels=target,
      #                                 return_dict = True)

       # captioning_features = output_dictionary_captioning.hidden_states[12][:,0,:]

            
        
        '''
        Compute Classification Output and loss from Fusion model
        '''
        output = fusion_model(text_features)
        
        target = target.float()
        loss = criterion(output, target.unsqueeze(1))

            
        test_loss.extend([loss.item()]*feats.size()[0])
        
        
        
        '''
        Prediction
        '''
        
        predictions = F.softmax(output, dim=1)
        
        _, top1_pred_labels = torch.max(predictions,1)
        top1_pred_labels = top1_pred_labels.view(-1)
        
        top1_accuracy += torch.sum(torch.eq(top1_pred_labels, target)).item()
        
        
        total += len(target)
        
        #del feats
        del captions
        del input_ids
        del attention_masks
        #del input_ids_cap
        #del attention_masks_cap
        del target
        del loss
            
    fusion_model.train()
    return np.mean(test_loss), top1_accuracy/total


In [ ]:
writer = SummaryWriter(model_name)

train_s(Text_model, Fusion_model, train_dataloader, validation_dataloader, criterion, optimizer, lr_scheduler, model_path, writer, device, epochs = num_Epochs)

writer.flush()
writer.close()

Streaming output truncated to the last 5000 lines.
okay  1285
okay  1286
okay  1287
okay  1288
okay  1289
okay  1290
okay  1291
okay  1292
okay  1293
okay  1294
okay  1295
okay  1296
okay  1297
okay  1298
okay  1299
okay  1300
okay  1301
okay  1302
okay  1303
okay  1304
okay  1305
okay  1306
okay  1307
okay  1308
okay  1309
okay  1310
okay  1311
okay  1312
okay  1313
okay  1314
okay  1315
okay  1316
okay  1317
okay  1318
okay  1319
okay  1320
okay  1321
okay  1322
okay  1323
okay  1324
okay  1325
okay  1326
okay  1327
okay  1328
okay  1329
okay  1330
okay  1331
okay  1332
okay  1333
okay  1334
okay  1335
okay  1336
okay  1337
okay  1338
okay  1339
okay  1340
okay  1341
okay  1342
okay  1343
okay  1344
okay  1345
okay  1346
okay  1347
okay  1348
okay  1349
okay  1350
okay  1351
okay  1352
okay  1353
okay  1354
okay  1355
okay  1356
okay  1357
okay  1358
okay  1359
okay  1360
okay  1361
okay  1362
okay  1363
okay  1364
okay  1365
okay  1366
okay  1367
okay  1368
okay  1369
okay  1370
oka

AttributeError: ignored

In [ ]:
len(svm_t)

6283

In [ ]:
final_t = svm_t[0].cpu().numpy()
for i in range(1,395):
  g = svm_t[i].cpu().numpy()
  final_t = np.append(final_t,g)

In [ ]:
final_1_t = svm_train[0].cpu().detach().numpy()
for i in range(1,395):
  g = svm_train[i].cpu().detach().numpy()
  final_1_t = np.append(final_1_t,g,axis=0)
  print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
final = svm_t[0].cpu().numpy()
for i in range(1,6283):
  g = svm_t[i].cpu().numpy()
  final = np.append(final,g)

In [ ]:
final_1 = svm_train[0].cpu().detach().numpy()
for i in range(1,6283):
  g = svm_train[i].cpu().detach().numpy()
  final_1 = np.append(final_1,g,axis=0)
  print(i)

Streaming output truncated to the last 5000 lines.
1283
1284
1285
1286
1287
1288
1289
1290
1291
1292
1293
1294
1295
1296
1297
1298
1299
1300
1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
1345
1346
1347
1348
1349
1350
1351
1352
1353
1354
1355
1356
1357
1358
1359
1360
1361
1362
1363
1364
1365
1366
1367
1368
1369
1370
1371
1372
1373
1374
1375
1376
1377
1378
1379
1380
1381
1382
1383
1384
1385
1386
1387
1388
1389
1390
1391
1392
1393
1394
1395
1396
1397
1398
1399
1400
1401
1402
1403
1404
1405
1406
1407
1408
1409
1410
1411
1412
1413
1414
1415
1416
1417
1418
1419
1420
1421
1422
1423
1424
1425
1426
1427
1428
1429
1430
1431
1432
1433
1434
1435
1436
1437
1438
1439
1440
1441
1442
1443
1444
1445
1446
1447
1448
1449
1450
1451
1452
1453
1454
1455
1456
1457
1458
1459
1460
1461
1462
1463
1464
1465
1466
1467
1468
1469
1470
1471
1472

In [ ]:
from sklearn.svm import SVC
clf = SVC(C=10000.0, random_state=1, kernel='rbf') 
clf.fit(final_1,final)

In [ ]:
final.size

26668

In [ ]:
c=0
ans = []
for i in range(0,26600):
  a=clf.predict(final_1[i].reshape(-1,512))
  ans.append(a)
  if a == final[i]:
    c = c+1
  print(i)

Streaming output truncated to the last 5000 lines.
21600
21601
21602
21603
21604
21605
21606
21607
21608
21609
21610
21611
21612
21613
21614
21615
21616
21617
21618
21619
21620
21621
21622
21623
21624
21625
21626
21627
21628
21629
21630
21631
21632
21633
21634
21635
21636
21637
21638
21639
21640
21641
21642
21643
21644
21645
21646
21647
21648
21649
21650
21651
21652
21653
21654
21655
21656
21657
21658
21659
21660
21661
21662
21663
21664
21665
21666
21667
21668
21669
21670
21671
21672
21673
21674
21675
21676
21677
21678
21679
21680
21681
21682
21683
21684
21685
21686
21687
21688
21689
21690
21691
21692
21693
21694
21695
21696
21697
21698
21699
21700
21701
21702
21703
21704
21705
21706
21707
21708
21709
21710
21711
21712
21713
21714
21715
21716
21717
21718
21719
21720
21721
21722
21723
21724
21725
21726
21727
21728
21729
21730
21731
21732
21733
21734
21735
21736
21737
21738
21739
21740
21741
21742
21743
21744
21745
21746
21747
21748
21749
21750
21751
21752
21753
21754
21755
21756
21757
2

In [ ]:
i

26599

In [ ]:
c

22556

In [ ]:
c

24338

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
clf = GradientBoostingClassifier(n_estimators=10000, learning_rate=0.8,max_depth=5, random_state=0)
clf.fit(final_1, final)

GradientBoostingClassifier(learning_rate=0.8, max_depth=5, n_estimators=10000,
                           random_state=0)

In [ ]:
from sklearn.metrics import accuracy_score  

In [ ]:
Y_pred = clf.predict(final_1)  
score = accuracy_score(final, Y_pred)  
print(score)  

0.904540209303251
